In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical  
from keras.datasets import cifar100
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout
import numpy as np

# (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))

# centre the data
def centre_data(train, test, validation):
    
    # calculate the means for each attribute of the training data
    column_means = np.mean(train, axis=0) 
    
    # centre training data by subtracting training data attribute means
    for i in range(len(train)):
        train[i] = train[i] - column_means
    
    # centre testing data by subtracting training data attribute means
    for x in range(len(test)):
        test[x] = test[x] - column_means
        
    # centre testing data by subtracting training data attribute means
    for x in range(len(validation)):
        validation[x] = validation[x] - column_means
        
    return train, test, validation

# helper function for concatenating labels onto their corresponding data points
def concatenate_data(training_data, training_labels):
    return np.column_stack((training_data, training_labels))

# apply PCA on the data 
def PCA(variance_target, training_data, testing_data, validation_data):

    U, sigma, Vt = np.linalg.svd(training_data, full_matrices=False)
    
    sum_square_singular = np.sum(sigma**2)
    
    ratios = sigma**2/sum_square_singular
    
                
    n_components = 0
    explained_variance = 0
    
    # determine how many principle components must be retained to maintain the target level of explained variance
    for i in range(len(ratios)):
        if explained_variance >= variance_target:
            break
        else: 
            n_components += 1
            explained_variance += ratios[i]
    
    return training_data.dot(Vt.T[:, :n_components]), testing_data.dot(Vt.T[:, :n_components]), validation_data.dot(Vt.T[:, :n_components])


# data set is randomised and then split in a 70:30 ratio for training:validation sets
def split_into_validation_training(training_matrix):
    
    import random
    random.shuffle(training_matrix)

    training_set = training_matrix[:int(len(training_matrix)*0.7)]
    validation_set = training_matrix[int(len(training_matrix)*0.7):]
    
    return training_set, validation_set

def load_in_dataset_and_preprocess(explained_variance):
    (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))
    
    # reshape the data 
    training_data = training_data.reshape(50000, 3072)
    testing_data = testing_data.reshape(10000, 3072)
    
    concatenated_training = concatenate_data(training_data, training_labels)
    
    training_set, validation_set = split_into_validation_training(concatenated_training)
    
    training_data = training_set[:, :-1]
    training_labels = np.squeeze(training_set[:, -1])
    
    validation_data = validation_set[:, :-1]
    validation_labels = np.squeeze(validation_set[:, -1])
    
    training_data = training_data.astype('float32')
    testing_data = testing_data.astype('float32')
    validation_data = validation_data.astype('float32')

#     # Centre data
#     training_data, testing_data, validation_data = centre_data(training_data, testing_data, validation_data)

#     # Apply PCA
#     training_data, testing_data, validation_data = PCA(explained_variance, training_data, testing_data, validation_data)
    
    # Normalization of pixel values (to [0-1] range)
    training_data = training_data / 255
    testing_data = testing_data / 255
    validation_data = validation_data / 255
    
    return training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels

# training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)


In [28]:
# training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)

In [2]:
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout

def run_MLP_model(training_data, training_labels, testing_data, testing_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay_level, momentum, epochs, loss_function):

    model = Sequential()

    model.add(Dense(num_hidden_units, activation=first_activation_function, input_dim=training_data.shape[1]))
    model.add(Dropout(0.5))
    model.add(Dense(num_hidden_units, activation=first_activation_function))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation=second_activation_function))

    if (optimiser == 'SGD'):
        op = SGD(lr=learning_rate, decay=decay_level, momentum=momentum, nesterov=True)

    else:
        op = Adam(lr=learning_rate, decay=decay_level)

    # can also use loss function categorical_crossentropy
    # or optimiser SGD
    # try with different optimisers and loss functions
    model.compile(optimizer=op,
                  loss=loss_function,
                  metrics=['accuracy'])

    history = model.fit(training_data, training_labels, epochs=epochs, batch_size=32, verbose=0, validation_split=0.2)

    score = model.evaluate(testing_data, testing_labels, batch_size=128, verbose=0)
    
    return score[1]

In [30]:
# score = run_MLP_model(training_data, training_labels, validation_data, validation_labels, 'relu', 'softmax', 256, 0.001, 'Adam', 1e-6, 0.9, 20, 'sparse_categorical_crossentropy')
# print(score[1])

In [31]:
### Converting images to grayscale

In [32]:
def convert_to_grayscale(train, test):
    rgb_weights = [0.2989, 0.5870, 0.1140]
    gray_train = np.dot(train[:][...,:3], rgb_weights)
    gray_test = np.dot(test[:][...,:3], rgb_weights)
    return gray_train.astype(int), gray_test.astype(int)

In [33]:
### Increasing image contrast

In [34]:
def increase_image_contrast(image, strength):
    p_low, p_high = np.percentile(image, (strength, 100 - strength))
    return (exposure.rescale_intensity(image, in_range=(p_low, p_high))).astype(int)

def increase_all_contrast(train, test, strength=10):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = increase_image_contrast(train[i], strength)
    for i in range(len(test)):
        new_test[i] = increase_image_contrast(test[i], strength)
    return new_train, new_test

In [35]:
### Image equalisation

In [36]:
def equalize_hist(image):
    return (exposure.equalize_hist(image) * 255).astype(int)

def equalize_hist_all(train, test):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = equalize_hist(train[i])
    for i in range(len(test)):
        new_test[i] = equalize_hist(test[i])
    return new_train, new_test



In [37]:
def equalize_adapthist(image, clip_lim):
    return (exposure.equalize_adapthist(image, clip_limit=clip_lim) * 255).astype(int)

def equalize_adapthist_all(train, test, clip_limit=0.03):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = equalize_adapthist(train[i], clip_limit)
    for i in range(len(test)):
        new_test[i] = equalize_adapthist(test[i], clip_limit)
    return new_train, new_test

In [38]:
### Edge detection

In [39]:
def canny_edge_filter_all(train, test, sig=1):
    if (len(train.shape) > 3):
        train, test = convert_to_grayscale(train, test)
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = feature.canny(train[i].astype(float), sigma=sig)
    for i in range(len(test)):
        new_test[i] = feature.canny(test[i].astype(float), sigma=sig)
    return new_train, new_test

In [40]:
### flattening the images

In [41]:
def flatten_data(x_train, y_train, x_test, y_test):
    new_image_shape = 1
    for dim in range(1, len(x_train.shape)):
        new_image_shape *= x_train.shape[dim]
        
    flat_x_train = x_train.reshape((x_train.shape[0], new_image_shape))
    flat_y_train = np.ravel(y_train)
    
    flat_x_test = x_test.reshape((x_test.shape[0], new_image_shape))
    flat_y_test = np.ravel(y_test)
    return flat_x_train, flat_y_train, flat_x_test, flat_y_test

In [42]:
### Testing

In [4]:
import time
import csv

In [5]:
def nano_to_seconds(nanoseconds):
    """Converts nanoseconds to seconds rounded to the nearest 5 decimal places.
    
    Parameters
    ----------
    nanoseconds : int
        The nanoseconds to convert
    """

    return np.round((nanoseconds / 1e+9), 5)

In [45]:
## test preprocessing

In [46]:
## test second activation functions

In [47]:

def test_MLP_second_activation_functions():
    activation_functions = ['logistic', 'tanh', 'softmax']
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_second_activation_function.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for activation_function in activation_functions:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

# test_MLP_second_activation_functions()

In [48]:
## test first activation functions

In [49]:
def test_MLP_first_activation_functions():
    activation_functions = ['elu', 'linear', 'relu', 'tanh']
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_first_activation_function.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for activation_function in activation_functions:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

# test_MLP_first_activation_functions()

In [50]:
## test learning rates

In [51]:
def test_MLP_learning_rates():
    learning_rates = [0.01, 0.05, 0.001]
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_learning_rates.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for lr in learning_rates:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, lr, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, lr, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_learning_rates()

NameError: name 'activation_function' is not defined

In [ ]:
## test number of hidden units

def test_MLP_hidden_units():
    hidden_units_numbers = [50, 256, 500]
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_hidden_units.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for hu in hidden_units_numbers:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, hu, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, hu, avg_accuracy, avg_runtime])

# test_MLP_hidden_units()

In [ ]:
## test decay

def test_MLP_decay_rates():
    decay_rates = [0.00001, 0.000001, 0.0000001]
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_decay_rates.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for dr in decay_rates:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, dr, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, dr, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_decay_rates()

In [ ]:
## test momentum 
def test_MLP_momentums():
    momentums = [0.5, 0.7, 0.9]
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_momentums.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
#         momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'SGD'
        epochs = 20
        num_hidden_units = 256
        for momentum in momentums:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_momentums()

In [ ]:
## test epochs

def test_MLP_epochs():
    diff_epochs = [20, 50, 100]
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_epochs.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
#         epochs = 20
        num_hidden_units = 256
        for epochs in diff_epochs:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_epochs()

In [ ]:
## test different optimisers 

optimisers = ['Adam', 'SGD']

def test_MLP_optimisers():
    optimisers = ['Adam', 'SGD']
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_optimisers.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
#         optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for optimiser in optimisers:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_optimisers()

In [10]:
# test different loss functions 

def test_MLP_loss_functions():
    loss_functions = ['categorical_crossentropy']
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    with open('mlp_loss_functions2.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
#         loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        training_labels = to_categorical(training_labels, 20)
        testing_labels = to_categorical(testing_labels, 20)
        for loss in loss_functions:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_loss_functions()

ValueError: in user code:

    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1208 run_step  **

In [ ]:
# test different loss functions 

def test_MLP_contrast_stretching_functions():
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    stretch_factors = list(range(0, 46, 5))
    with open('mlp_contrast_stretching.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for stretch in stretch_factors:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            training_data, testing_data = increase_all_contrast(training_data, validation_data, strength)
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_contrast_stretching_functions()

In [ ]:
# test different loss functions 
def test_MLP_preprocessing_functions():
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    techniques = ['none', 'grayscale', 'hist equalization', 'adaptive equalization', 'canny']
    with open('mlp_preprocessing.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for technique in techniques:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
             if technique == 'grayscale':
                training_data, validation_data = convert_to_grayscale(training_data, validation_data)
            elif technique == 'hist equalization':
                training_data, validation_data = equalize_hist_all(training_data, validation_data)
            elif technique == 'adaptive equalization':
                training_data, validation_data = equalize_adapthist_all(training_data, validation_data)
            elif technique == 'canny':
                training_data, validation_data = canny_edge_filter_all(training_data, validation_data, 0)
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_preprocessing_functions()

In [ ]:
# test different loss functions 

### UNCOMMENT PCA CALL IN LOAD IN DATA BEFORE RUNNING THIS

def test_MLP_PCA():
    explained_variances = [0.6, 0.7, 0.8, 0.9]
    with open('mlp_pca.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for explained_variance in explained_variances:
            training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(explained_variance)
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_PCA()

In [ ]:
# test different loss functions 

### UNCOMMENT PCA AND CENTRE CALL IN LOAD IN DATA BEFORE RUNNING THIS

def test_MLP_PCA_centre():
    explained_variances = [0.6, 0.7, 0.8, 0.9]
    with open('mlp_pca_centre.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for explained_variance in explained_variances:
            training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(explained_variance)
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_PCA_centre()

In [ ]:
## JUST COMMENT OUT CENTRE CALL AND RUN, THEN ADD IT IN AND RUN

def test_MLP_centre():
    explained_variances = [0.9]
    with open('mlp_centre.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Units', 'Average Accuracy', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 20
        num_hidden_units = 256
        for explained_variance in explained_variances:
            training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(explained_variance)
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, validation_data, validation_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, num_hidden_units, avg_accuracy, avg_runtime])

test_MLP_centre()